In [1]:
import matplotlib.pyplot as plt
import scipy.stats as stats
from pyddm import Model
from pyddm import Sample
import pyddm as ddm
import numpy as np
import pandas as pd
import random
import os
import copy

In [3]:
[5]*10

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

Like the DDM model, but with multiple particles, where decisions about absence require all particles to hit the lower boundary, but decisions about presence require one particle to hit the upper coundary.

## Simulate the drift diffusion model

This is the most general model: it allows the upper and lower boundaries to collapse at different times and with different slopes, and for the drift rates for target presence and absence to have different means and standard deviations.

In [4]:
class SubjDDM:
    
    def __init__(self, subj_id, v, n_particles, a, z, t0, tmax, sigma, collapse_time, collapse_rate, dt):
        """
        Initializes the drift diffusion model with the given parameters.

        Args:
        - subj_id: subject identifier
        - v: drift rates for target absence and presence
        - a: boundary
        - n_particles: number of particles
        - z: starting point bias (in absolute units, i.e. not scaled)
        - t0: non-decision time
        - tmax: maximum trial duration (not including t0)
        - sigma: standard deviations of the Gaussian noise for target absence and presence
        - collapse_time: the time in which the lower and upper boundaries start collapsing
        - collapse_rate: the speed with which the lower and upper boundaries collapse
        - dt: time step size
        """
        self.subj_id = subj_id
        self.v = v
        self.n_particles = n_particles
        self.a = [-a,a] #lower and upper boundaries
        self.z = z
        self.t0 = t0
        self.tmax = tmax
        self.sigma = sigma
        self.collapse_time = collapse_time
        self.collapse_rate = collapse_rate
        self.dt = dt
    

    def simulate(self, n_trials):
        """
        Simulates the drift diffusion model for the given number of trials.

        Args:
        - n_trials: number of trials to simulate

        Returns:
        - A pandas DataFrame with columns "present", "decision", "rt" (response time) and "correct"
        """
        
        df = pd.DataFrame(columns=["subj_id", "present", "decision","rt", "correct"])

        for i in range(n_trials):
            
            present = random.randrange(2)
            v = self.v[present]
            s = self.sigma[present]
            a = copy.copy(self.a)            
            xs = self.n_particles*[self.z]
            t = 0.0
            
            while (all([a[0] < x < a[1] for x in xs])) & (t<self.tmax):
                
                xs = [x + v*self.dt + s*np.sqrt(self.dt)*random.normalvariate(0,1) for x in xs]
                
                if t>self.collapse_time[0]:
                    a[0]+=self.collapse_rate[0]*self.dt
                if t>self.collapse_time[1]:
                    a[1]+=self.collapse_rate[1]*self.dt
                    
                t = t + self.dt

            if (any([x >= a[1] for x in xs])) & (t<self.tmax-self.t0):
                decision = 1
                correct = decision==present
                rt = t + self.t0
                df.loc[i] = [self.subj_id, present,decision,rt, correct]
            elif (all([x<= a[0] for x in xs])) & (t<self.tmax-self.t0):
                decision = 0
                correct = decision==present
                rt = t + self.t0
                df.loc[i] = [self.subj_id, present,decision,rt, correct]
            else:
                continue
            
        return(df)
    
class GroupDDM:
    
    def __init__(self, v, a, z, t0, tmax, sigma, collapse_time, collapse_rate, dt):
        """
        Initializes the drift diffusion model with the given parameters.

        Args:
        - v: probability distributions for drift rates for target absence and presence
        - a: probability distributions for boundary
        - z: probability distributions for starting point bias (in absolute units, i.e. not scaled)
        - t0: probability distributions for non-decision time
        - tmax: maximum trial duration (not including t0)
        - sigma: probability distributions for standard deviations of the Gaussian noise for target absence and presence
        - collapse_time: probability distributions for the time in which the lower and upper boundaries start collapsing
        - collapse_rate: probability distributions for the speed with which the lower and upper boundaries collapse
        - dt: time step size
        """
        self.v = v
        self.a = a
        self.z = z
        self.t0 = t0
        self.tmax = tmax
        self.sigma = sigma
        self.collapse_time = collapse_time
        self.collapse_rate = collapse_rate
        self.dt = dt
        self.df = pd.DataFrame(columns=['subj_id', 'v_absent', 'v_present', 'a',
                                       'z', 't0', 'tmax', 'sigma_absent', 'sigma_present', 'ct_absent',
                                       'ct_present', 'cr_absent', 'cr_present', 'dt'])
        
    def simulate(self, n_subj, n_trials):
        """
        Simulates the drift diffusion model for the given number of subjects and trials.

        Args:
        - n_subj: number of subjects to simulate
        - n_trials: number of trials to simulate

        Returns:
        - A pandas DataFrame with columns "present", "decision", "rt" (response time) and "correct"
        """
        
        self.generated_data = pd.DataFrame()
        
        def sample_value(x):
            
            if (type(x)==float) | (type(x)==int):
                return x
            elif (type(x)==list):
                values = [];
                for a in x:
                    values.append(sample_value(a))
                return(values)
            else:
                return x.rvs()
            
            
        
        for i in range(n_subj):
            
            v_s = sample_value(self.v)
            a_s = sample_value(self.a)
            z_s = sample_value(self.z)
            t0_s = sample_value(self.t0)
            sigma_s = sample_value(self.sigma)
            collapse_time_s = sample_value(self.collapse_time)
            collapse_rate_s = sample_value(self.collapse_rate)
            
            self.df = self.df.append({'subj_id': i,
                            'v_absent': v_s[0], 
                            'v_present': v_s[1], 
                            'a': a_s, 
                            'z': z_s,
                            't0': t0_s,
                            'tmax': self.tmax, 
                            'sigma_absent': sigma_s[0], 
                            'sigma_present': sigma_s[1], 
                            'ct_absent': collapse_time_s[0],
                            'ct_present': collapse_time_s[1],
                            'cr_absent': collapse_rate_s[0], 
                            'cr_present': collapse_rate_s[1], 
                            'dt': self.dt}, ignore_index=True)
            
            subj = SubjDDM(i, v_s, a_s, z_s, t0_s, self.tmax, sigma_s, collapse_time_s, collapse_rate_s, self.dt)
            self.generated_data = pd.concat( [self.generated_data, subj.simulate(n_trials)])
            
            
            

In [18]:
random.seed(1)
Ntrials = 10000
#collapsing boundaries, drift rate for absence is 0
test = SubjDDM(subj_id = 1,
             v=[-0.2,0.2],
             n_particles=2,
               a = 1, 
               z = 0, 
               t0 = 0.3, 
               tmax =5, 
               sigma = [1,1],
               collapse_time = [0,0],
               collapse_rate = [0,0], 
               dt=0.01)
test_df = test.simulate(Ntrials)
test_df.to_csv('data/generated_data/test_generated.csv', index=False)